# Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Funções

In [37]:
def cria_modelos(df, df_pred, x_test, y_test):
    '''
    Função que cria modelos para a previsão
    '''
    X = df[x_test]
    y = df[y_test]

    model = LinearRegression()
    model.fit(X,y)
    
    price_pred = model.predict(df_pred[x_test])
    
    return price_pred

# Read dataset

In [2]:
diamonds_test = pd.read_csv('data/diamonds.csv')
diamonds_rick = pd.read_csv('data/rick_diamonds.csv')
diamonds_test

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
1,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
2,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
3,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
4,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
...,...,...,...,...,...,...,...,...,...,...
48935,0.72,Premium,D,SI1,62.7,59.0,2757,5.69,5.73,3.58
48936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
48937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
48938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56


# Data Cleaning 

In [3]:
# removendo as colunas de x,y,z que estiverem todas zeradas
diamonds_test = diamonds_test[~((diamonds_test['x'] == 0) & (diamonds_test['y'] == 0) & (diamonds_test['z'] == 0))]

In [4]:
# remoção das colunas com x =0, como todas nessas colunas y ou z também era =0
diamonds_test = diamonds_test[~(diamonds_test['x'] == 0)]

In [5]:
# não havia colunas y = 0, logo remoção desnecessária
diamonds_test[diamonds_test['y'] == 0]

,carat,cut,color,clarity,depth,table,price,x,y,z


In [6]:
# completando as informações perdidas quando o z = 0
diamonds_test.loc[diamonds_test['z']==0 ,'z'] = (diamonds_test['depth']*(diamonds_test['x']+diamonds_test['y']))/2

In [7]:
# buscando valores faltando em outras colunas
diamonds_test[diamonds_test['price'] == 0]

,carat,cut,color,clarity,depth,table,price,x,y,z


In [8]:
diamonds_test[diamonds_test['table'] == 0]

,carat,cut,color,clarity,depth,table,price,x,y,z


In [9]:
diamonds_test['clarity'].unique()

array(['SI1', 'VS1', 'VS2', 'SI2', 'VVS2', 'VVS1', 'I1', 'IF'],
      dtype=object)

In [10]:
diamonds_test['color'].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [11]:
diamonds_test['cut'].unique()

array(['Premium', 'Good', 'Very Good', 'Ideal', 'Fair'], dtype=object)

# Exploratory Analysis

In [12]:
# identificando as correlações
diamonds_test.corr()['price']
#outras explorações foram feitas no tableau

carat    0.922207
depth   -0.012077
table    0.127854
price    1.000000
x        0.887904
y        0.866568
z        0.112812
Name: price, dtype: float64

# Data transformation

# Modeling

In [32]:
#Criando a Coluna price_predicted no dataframe
diamonds_rick['price_predicted'] = 0 

In [48]:
#criando modelo para Clarity = VVS1, VVS2 e IF e aplicando no diamonds_rick
mask_teste = (diamonds_test['clarity'] == 'VVS1') |(diamonds_test['clarity'] == 'VVS2') | (diamonds_test['clarity'] == 'IF')
mask_rick = (diamonds_rick['clarity'] == 'VVS1') |(diamonds_rick['clarity'] == 'VVS2') | (diamonds_rick['clarity'] == 'IF')

diamonds_rick.loc[mask_rick, 'price_predicted'] = cria_modelos(diamonds_test[mask_teste],diamonds_rick[mask_rick],['carat'],'price')

In [49]:
#criando modelo para Clarity = VS1 e VS2 e e aplicando no diamonds_rick
mask_teste = (diamonds_test['clarity'] == 'VS1') |(diamonds_test['clarity'] == 'VS2')
mask_rick = (diamonds_rick['clarity'] == 'VS1') |(diamonds_rick['clarity'] == 'VS2')

diamonds_rick.loc[mask_rick, 'price_predicted'] = cria_modelos(diamonds_test[mask_teste],diamonds_rick[mask_rick],['carat'],'price')

In [50]:
#criando modelo para Clarity = SI1 e SI2 e e aplicando no diamonds_rick
mask_teste = (diamonds_test['clarity'] == 'SI1') |(diamonds_test['clarity'] == 'SI2')
mask_rick = (diamonds_rick['clarity'] == 'SI1') |(diamonds_rick['clarity'] == 'SI2')

diamonds_rick.loc[mask_rick, 'price_predicted'] = cria_modelos(diamonds_test[mask_teste],diamonds_rick[mask_rick],['carat'],'price')

In [51]:
#criando modelo para Clarity = I1 e e aplicando no diamonds_rick
mask_teste = (diamonds_test['clarity'] == 'I1')
mask_rick = (diamonds_rick['clarity'] == 'I1')

diamonds_rick.loc[mask_rick, 'price_predicted'] = cria_modelos(diamonds_test[mask_teste],diamonds_rick[mask_rick],['carat'],'price')

# To CSV

In [52]:
diamonds_rick.to_csv('testes_price/price_pred', index=False)
diamonds_rick

,carat,cut,color,clarity,depth,table,x,y,z,price_predicted
0,0.91,Very Good,H,SI2,63.4,57.0,6.10,6.18,3.89,4169.858678
1,0.38,Good,F,VS2,60.3,61.0,4.68,4.71,2.83,660.426324
2,0.52,Ideal,H,VS2,61.8,56.0,5.19,5.16,3.20,1884.709095
3,1.10,Ideal,G,VS1,61.3,56.0,6.65,6.69,4.09,6956.737718
4,2.28,Ideal,G,SI2,61.6,57.0,8.44,8.37,5.18,14876.070047
...,...,...,...,...,...,...,...,...,...,...
4995,0.30,Very Good,D,SI2,62.2,59.0,4.25,4.30,2.66,-597.140545
4996,0.51,Ideal,F,VVS2,61.9,56.0,5.09,5.14,3.16,2548.555092
4997,1.01,Very Good,F,SI1,60.6,60.0,6.37,6.43,3.88,4951.333961
4998,0.31,Ideal,F,VVS1,62.1,56.0,4.35,4.38,2.71,475.724473


# Results


In [15]:
# erro usando a média foi 3980.71
# erro usando a mediana foi 4255.53
# erro usando a primeira modelagem foi 1605.15
# erro após a limpeza dos dados foi  1605.13
# erro usando segundo modelo foi 1578.53
# erro usando mais modelos foi 1254.41